# Cow behaviour labelling
Using a pre-trained model.

In [1]:
import os

notebook_mode: int = int(
    input(
        """
    Select notebook mode: 
    1. Google Colab  2. Local
    """
    )
)

if notebook_mode == 1:
    # Run on Colab.
    INPUT_PATH: str = "/content/drive/MyDrive/Ellinbank/video_observation/data/"
    SCRIPT_PATH: str = "/content/drive/MyDrive/Ellinbank/video_observation/training_testing/data_labelling/"
    OUTPUT_PATH: str = "/content/drive/MyDrive/Ellinbank/video_observation/output/"
    os.system(command="cp {}custom_model.py .".format(SCRIPT_PATH))
    os.system(command="cp {}inference.py .".format(SCRIPT_PATH))
    os.system(command="cp {}utils.py .".format(SCRIPT_PATH))
    os.system(command="cp {}operation.py .".format(SCRIPT_PATH))
elif notebook_mode == 2:
    INPUT_PATH: str = "../../../../data/"
    SCRIPT_PATH: str = "./"
    OUTPUT_PATH: str = "./out/"

In [2]:
import pandas as pd


files: list[str] = os.listdir(path=INPUT_PATH)
files = [f for f in files if f.endswith(".zip")]

sensor_names: list[str] = [name.split("_")[0] for name in files]
print(sensor_names)

['MOS2E03230475']


In [3]:
import tensorflow as tf

print(tf.__version__)

training_device: str = "cpu"

if str.lower(training_device) == "cpu":
    tf.config.set_visible_devices(
        devices=[],
        device_type="GPU",
    )
else:
    physical_devices = tf.config.experimental.list_physical_devices("GPU")
    assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
    config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

2024-01-04 04:17:54.515406: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-04 04:17:54.663836: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-04 04:17:54.663878: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-04 04:17:54.672940: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-04 04:17:54.694843: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-04 04:17:54.695607: I tensorflow/core/platform/cpu_feature_guard.cc:1

2.15.0


In [4]:
import random

import keras

window_size: int = 600  # 300: 10 seconds
window_per_epoch: int = 200
epoch: int = 1
batch_size: int = 64
# random.seed(715) # 715 looks good.
random.seed(785)  # 785 makes "other" looks bad, otherwise is good.


model: keras.Model = keras.models.load_model(
    filepath="{}cow_model".format(SCRIPT_PATH)
)
model.summary()

Model: "cow_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             multiple                  13184     
                                                                 
 conv1d_1 (Conv1D)           multiple                  24640     
                                                                 
 conv1d_2 (Conv1D)           multiple                  6176      
                                                                 
 batch_normalization (Batch  multiple                  512       
 Normalization)                                                  
                                                                 
 conv1d_transpose (Conv1DTr  multiple                  1552      
 anspose)                                                        
                                                                 
 conv1d_transpose_1 (Conv1D  multiple                  39

In [5]:
# for f in files:
#     data_chunks: pd.DataFrame = pd.read_csv(
#         filepath_or_buffer="{}/{}".format(INPUT_PATH, f),
#     )

# data_chunks.info()

In [6]:
import gc
from pathlib import Path

import numpy as np
import pandas.io.parsers.readers
from operation import clean_data, label_data

saved_df: pd.DataFrame = ...
saved_labels: np.ndarray = ...

for f in files:
    os.system("rm -rf {}{}.txt".format(OUTPUT_PATH, f.split("_")[0]))
    os.system("rm -rf {}{}_timestamps.txt".format(OUTPUT_PATH, f.split("_")[0]))
    Path("{}{}.txt".format(OUTPUT_PATH, f.split("_")[0])).touch()
    data_chunks: pandas.io.parsers.readers.TextFileReader = pd.read_csv(
        filepath_or_buffer="{}/{}".format(INPUT_PATH, f),
        # nrows=14000,
        chunksize=window_size * 10,
    )

    for raw_data in data_chunks:
        full_data: pd.DataFrame = clean_data(full_data=raw_data)
        labels: np.ndarray = label_data(
            raw_data=full_data,
            model=model,
            window_size=window_size,
            batch_size=batch_size,
        )

        saved_df = full_data
        saved_labels = labels

        with open(
            file="{}{}.txt".format(OUTPUT_PATH, f.split("_")[0]),
            mode="ab",
        ) as file:
            np.savetxt(
                fname=file,
                X=labels,
                fmt="%s",
            )

        time_stamps: np.ndarray = full_data["timestamps"].to_numpy()
        with open(
            file="{}{}_timestamps.txt".format(OUTPUT_PATH, f.split("_")[0]),
            mode="ab",
        ) as file:
            np.savetxt(
                fname=file,
                X=time_stamps,
                fmt="%s",
            )

        gc.collect()

In [7]:
overlapping_size: int = window_size - (len(saved_df) - saved_labels.size)
overlapping_size

213

In [8]:
last_window: pd.DataFrame = saved_df[-600:]
overlapping_labels: np.ndarray = label_data(
    raw_data=last_window,
    model=model,
    window_size=window_size,
    batch_size=batch_size,
)[:overlapping_size]

overlapping_labels.size

213

In [9]:
saved_labels_overlap: np.ndarray = saved_labels[-overlapping_size:]
saved_labels_overlap.size

213

In [10]:
print(np.sum(overlapping_labels == saved_labels_overlap))

211
